##**Caso de estudio - Árboles de decisión**
Una fintech quiere la implemntación de un modelo de preaprobación de creditos utilizando un modelo de arbol de decisión. Las variables de entrada y salida se definen a continuación:
* Preaprobación (salida - referencia): Indica en una etapa muy temprana si le presto o no le presto a una persona.
* Edad (entrada) : Indica la edad que posee una persona al momento de solicitar un crédito.


1. se cargan las librerías de trabajo

In [1]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier  #modelo de arbol de decisión
from sklearn.metrics import confusion_matrix
#carga automática de las bases de datos
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


2. Se procede con la carga de los datos

In [4]:
nxl = '/content/drive/MyDrive/Analítica de Negocios/6. SolicitantesCrédito(USD).xlsx'
XDB = pd.read_excel(nxl , sheet_name= 1) #Base de datos AAA
XDB = XDB.dropna()  #Elimina celdas erroneas o N/A
XDB = XDB[['Edad' , 'Perscargo' , 'Ingresos' , 'Egresos' , 'Monto (EAD)' , 'PreApr']]
#Separa variables de entrada y variables de salida
XD = XDB[['Edad' , 'Perscargo' , 'Ingresos' , 'Egresos' , 'Monto (EAD)']] # variables entrada
yd = XDB['PreApr'] #variables de referencia - salida


,PreApr
0,1
1,1
2,0
3,1
4,1
...,...
5837,1
5838,1
5839,1
5840,1


3. se procede con la implementación del modelo

In [5]:
mar = DecisionTreeClassifier(criterion = 'gini' , max_depth = 4) #max depth corresponde el maximo nivel de profundidad que tiene el arbol (5 porque tengo 5 variables)
mar.fit(XD , yd)

#para la grafica del arbol
from sklearn.tree import export_graphviz
from pydotplus import graph_from_dot_data
dot_data = export_graphviz(mar , feature_names = XD.columns)
graph = graph_from_dot_data(dot_data)
graph.write_png('2. Tree_Clase.png')

True

4. evaluamos el comportamiento del modelo frente a los datos presentados

In [6]:
ydp = mar.predict(XD) #vamos a estresar el modelo - backtesting
cm = confusion_matrix(yd , ydp)
print(cm)
VN = cm[0,0]
FP = cm[0,1]
FN = cm[1,0]
VP = cm[1,1]
print('VN = ' , VN)
print('FP = ' , FP)
print('FN = ' , FN)
print('VP = ' , VP)
TD = VN + FP + FN + VP
Ex = (VN+VP)/TD ; print("exactitud: " , Ex) #comportamiento general del modelo
Ter= (FP+FN)/TD ; print("tasa de error: " , Ter) #comportamiento erroneo del modelo
Sen = VP/(VP+FN) ; print("sensibilidad: " , Sen) #comportamiento frente a los preapro
Esp= VN/(VN+FP) ; print("especificidad: " , Esp) #comportamiento frente a los prenegados

prec = VP/(VP+FP) ; print('Precisión' , prec) #comportamiento solamente contando los positivos

PredNeg = VN/(VN+FN)  ; print('Predictividad negativa' , PredNeg)


[[2301  658]
 [ 644 2239]]
VN =  2301
FP =  658
FN =  644
VP =  2239
exactitud:  0.7771311194796303
tasa de error:  0.22286888052036974
sensibilidad:  0.7766215747485259
especificidad:  0.7776275768840825
Precisión 0.772868484639282
Predictividad negativa 0.7813242784380305


##**Análisis de resultados
de acuerdo con los resultados obtenidos por el modelo podemos observar de manera general que el modelo logró una exactitud que se ubivó por encima del 75% en promedio (0,777131) , lo que indica el buen comportamiento general del modelo frente a la preaprobación de créditos en una fintech.
Este valor está por encima de los porcentajs que establesce la Superfinanciera para este tipo de modelos. De esta misma se puede observar que tanto la sensibilidad como la especificidad lograron valor por encima del 75% eb promedio, sin embargo se destaca que la especificidad logra un porcentaje de (0,7776..) estuvo por encima de la sensibilidad(0.7767...) lo que indica que el modelo tiene la leve tendencia a prenegar crèditos.
Se destaca en el arbol que las variables que tuvieron una mayor relevancia fueron las variables monto (EAD) seguidas de los ingresos. Se destaca igualmente que el arbol de decisiòn solo logrò un nodo puro el cual està definido por la siguiente regla del negocio:MONTo <= 322 AND INGRESOS >376 AND EGRESOS <= 685 AND INGRESOS >644. en donde la decisiòn se denota {10,0} lo que indica la tendencia de esta regla a prenegar crèditos con una tendencia del 100%